<span style="color: Gold"> 파인튜닝 - 추가 학습

미세조정 전후 비교

In [19]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.optim import Adam

test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["긍정", "부정"]

# 미세 조정전 모델(랜덤 초기화)
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_before = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME)
model_before.eval() # 평가모드
with torch.no_grad():
    for i ,review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors = 'pt')
        outputs = model_before(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()
        print(f'리뷰: {i+1} : {review}')
        print(f'예측: {labels[pred_label]} {probs[pred_label]}')
        print(f'긍정: {probs[0]}| 부정:{ probs[1]}')

c:\Users\SAMSUNG\miniconda3\envs\llm_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


리뷰: 1 : This movie is absolutely amazing! Best film I've ever seen.
예측: 부정 0.6883774399757385
긍정: 0.3116225600242615| 부정:0.6883774399757385
리뷰: 2 : Terrible waste of time. Worst movie ever made.
예측: 부정 0.7562088966369629
긍정: 0.24379108846187592| 부정:0.7562088966369629
리뷰: 3 : It was okay, nothing special but not bad either.
예측: 부정 0.6389191746711731
긍정: 0.3610808551311493| 부정:0.6389191746711731
리뷰: 4 : Brilliant acting and stunning visuals throughout!
예측: 부정 0.7219837307929993
긍정: 0.27801626920700073| 부정:0.7219837307929993
리뷰: 5 : Boring and predictable plot. Very disappointed.
예측: 부정 0.7179383039474487
긍정: 0.28206169605255127| 부정:0.7179383039474487


In [25]:

# 학습 - 미세조정
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
train_texts = [
    ("This is wonderful and fantastic!", 1),  # 긍정
    ("Absolutely terrible and awful!", 0),     # 부정
    ("I love this so much!", 1),
    ("Hated it completely!", 0)
]
BERT_MODEL_NAME = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(BERT_MODEL_NAME)
model_after = AutoModelForSequenceClassification.from_pretrained(BERT_MODEL_NAME,num_labels=2)
from transformers import AdamW
optimizer = AdamW(model_after.parameters(), lr=2e-5)
model_after.train()
for epoch in range(7):
    total_loss = 0
    for text,label in train_texts:
        optimizer.zero_grad()
        inputs = tokenizer(text,return_tensors='pt')
        labels = torch.tensor([label])
        outputs = model_after(**inputs,labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    avg_loss = total_loss / len(train_texts)
    print(f'epoch : {epoch+1}  loss : {avg_loss:.4f}')  

c:\Users\SAMSUNG\miniconda3\envs\llm_env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\SAMSUNG\miniconda3\envs\llm_env\lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable 

epoch : 1  loss : 0.7606
epoch : 2  loss : 0.6358
epoch : 3  loss : 0.5932
epoch : 4  loss : 0.4900
epoch : 5  loss : 0.3381
epoch : 6  loss : 0.2481
epoch : 7  loss : 0.2711


In [33]:

print(f'미세조정 후')
test_reviews = [
    "This movie is absolutely amazing! Best film I've ever seen.",
    "Terrible waste of time. Worst movie ever made.",
    "It was okay, nothing special but not bad either.",
    "Brilliant acting and stunning visuals throughout!",
    "Boring and predictable plot. Very disappointed."
]

labels = ["부정", "긍정"]

with torch.no_grad():
    for i, review in enumerate(test_reviews):
        inputs = tokenizer(review,return_tensors='pt')
        outputs =  model_after(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits,dim=-1)[0]
        pred_label = torch.argmax(probs).item()        
        print(f'리뷰 : {i+1} : {review}')
        print(f'예측 : {labels[pred_label]} {probs[pred_label]}')
        print(f'부정 : {probs[0]} | 긍정 : {probs[1]}\n')

미세조정 후
리뷰 : 1 : This movie is absolutely amazing! Best film I've ever seen.
예측 : 긍정 0.8291221261024475
부정 : 0.17087794840335846 | 긍정 : 0.8291221261024475

리뷰 : 2 : Terrible waste of time. Worst movie ever made.
예측 : 부정 0.7417399287223816
부정 : 0.7417399287223816 | 긍정 : 0.2582601010799408

리뷰 : 3 : It was okay, nothing special but not bad either.
예측 : 긍정 0.5480057001113892
부정 : 0.45199427008628845 | 긍정 : 0.5480057001113892

리뷰 : 4 : Brilliant acting and stunning visuals throughout!
예측 : 긍정 0.6976812481880188
부정 : 0.302318811416626 | 긍정 : 0.6976812481880188

리뷰 : 5 : Boring and predictable plot. Very disappointed.
예측 : 부정 0.6669453978538513
부정 : 0.6669453978538513 | 긍정 : 0.3330545425415039

